In [2]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import shap
from xgboost import XGBClassifier
# import random forest regressor
from sklearn.ensemble import RandomForestRegressor
#import linear regression
from sklearn.linear_model import LinearRegression
# import tqdm
from tqdm import tqdm
import tqdm
#import r2_score
from sklearn.metrics import r2_score
#import confusion matrix
from sklearn.metrics import confusion_matrix
# import roc auc score
from sklearn.metrics import roc_auc_score
from food_crisis_functions import *
import json
with open("forecasting_hyperparameters.json", "r") as file:
    best_params_xgb_regressor= json.load(file)
    
with open("forecasting_hyperparameters_p3.json", "r") as file:
    best_params_xgb_regressor_for_p3= json.load(file)

# read csv
df_nowcasting = pd.read_csv(r'C:\Users\swl00\IFPRI Dropbox\Weilun Shi\monitoring-food-crises\New_WorkSpace\1.Source Data\Nowcasting_Analysis_010825.csv')
df_forecasting = pd.read_csv(r'C:\Users\swl00\IFPRI Dropbox\Weilun Shi\monitoring-food-crises\New_WorkSpace\1.Source Data\Forecasting_Analysis_010825.csv')
#rename infra_index_m12 to infra_index_m12_l12, rename infra_index_s12 to infra_index_s12_l12
# add dummys for area_id and month year
#df = pd.concat([df, pd.get_dummies(df['area_id'], prefix='area_id')], axis=1)
#df = pd.concat([df, pd.get_dummies(df['date'], prefix='month_year')], axis=1)
# drop lat and lon
#df = df.drop(['lat', 'lon'], axis=1)
###drop fews_ipc_ha
#df = df.drop(['fews_ipc_ha'], axis=1)
# random split train and test
df_origin = df_forecasting.copy()
df = df_forecasting.copy()
y_pred_test = pd.DataFrame()
model_stats = pd.DataFrame()
#select phase1_percent is not na
df = df_origin[df_origin['phase1_percent'].notna()]
df_nowcasting = df_nowcasting[df_nowcasting['phase1_percent'].notna()]
# Sort by region and date
df = df.sort_values(by=['area_id', 'date'])
df_nowcasting = df_nowcasting.sort_values(by=['area_id', 'date'])
#drop overall phase
df = df.drop(['overall_phase'], axis=1)
#for each region, set last observation to be test set
# create a series of new outcome, phase2_worse=phase2_percent+phase3_percent+phase4_percent+phase5_percent, phase3_worse=phase3_percent+phase4_percent+phase5_percent, phase4_worse=phase4_percent+phase5_percent, phase5_worse=phase5_percent
df['phase2_worse'] = df['phase2_percent'] + df['phase3_percent'] + df['phase4_percent'] + df['phase5_percent']
df['phase3_worse'] = df['phase3_percent'] + df['phase4_percent'] + df['phase5_percent']
df['phase4_worse'] = df['phase4_percent'] + df['phase5_percent']
df['phase5_worse'] = df['phase5_percent']
#drop phase2_percent, phase3_percent, phase4_percent, phase5_percent, phase1_percent
df = df.drop(['phase2_percent', 'phase3_percent', 'phase4_percent', 'phase5_percent', 'phase1_percent'], axis=1)
# Splitting the data
#test_df = df.groupby('area_id').tail(1)
#train_df = df.drop(test_df.index)
#test_df = test_df.drop(['area_id','date'], axis=1)
#train_df = train_df.drop(['area_id','date'], axis=1)
y_pred_test = pd.DataFrame()
# drop anything after 2022-01-01
#df = df[df['date'] < '2021-01-01']
shape_values_df_ensemble = pd.DataFrame()
df_result = pd.DataFrame()
date = "2022-01-01"  # Define the 'date' variable
#order unique_dates
y_pred_test=pd.DataFrame()

diff_set = ['CPI',
 'fatalities_explosions_w5_m12',
 'GDP',
 'soil_moisture_mean_m12',
 'fatalities_explosions',
 'chirps_z_score',
 'EVI_m12',
 'event_count_battles_w10_m12',
 'event_count_battles_w5_m12',
 'price_index_s12',
 'event_count_explosions',
 'rainfall_chirps_s12',
 'event_count_violence_w10_m12',
 'WFP_Price_Change_s12',
 'WFP_Price_Volatility_m12',
 'WFP_Price_Change_m12',
 'infra_index_m12',
 'temperature_2m_mean_s12',
 'event_count_battles_m12',
 'fatalities_battles',
 'event_count_violence_m12',
 'event_count_explosions_m12',
 'fatalities_battles_w5_m12',
 'event_count_violence_w5_m12',
 'soil_moisture_mean_s12',
 'event_count_explosions_w5_m12',
 'WFP_Price_Volatility_s12',
 'event_count_violence',
 'precipitation_sum_m12',
 'event_count_violence_w10',
 'fatalities_violence_w10_m12',
 'event_count_battles',
 'event_count_explosions_w5',
 'fatalities_explosions_w10_m12',
 'event_count_battles_w10',
 'temperature_2m_mean_m12',
 'rainfall_chirps_m12',
 'fatalities_violence_m12',
 'WFP_Price_Change',
 'event_count_explosions_w10',
 'gini',
 'cpi',
 'fatalities_violence_w5_m12',
 'infra_index_s12',
 'shortwave_radiation_sum_m12',
 'price_index_m12',
 'fatalities_battles_w10_m12',
 'event_count_violence_w5',
 'fatalities_explosions_w10',
 'CC.PER.RNK',
 'fatalities_violence_w5',
 'shortwave_radiation_sum_s12',
 'fatalities_battles_w10',
 'fatalities_battles_m12',
 'nightlight_mean_m12',
 'fatalities_battles_w5',
 'event_count_battles_w5',
 'fatalities_violence_w10',
 'fatalities_violence',
 'infra_index',
 'fatalities_explosions_m12',
 'temperature_z_score',
 'event_count_explosions_w10_m12',
 'NY.GDP.PCAP.KD',
 'fatalities_explosions_w5',
 'precipitation_sum_s12',
 'WFP_Price_Volatility',
 'nightlight_mean_s12',
 'GOSIF_GPP_m12']




In [3]:
y_pred_test = pd.DataFrame()

for i in range(2, 6):
    train_df = df[df['date'] < date]
    test_df = df[df['date'] >= date]
    train_df = train_df.drop(['date','area_id'], axis=1)
    test_df = test_df.drop(['date','area_id'], axis=1)
    train_df_new = train_df.drop(['phase{}_worse'.format(j) for j in range(2, 6) if j != i], axis=1)
    test_df_new = test_df.drop(['phase{}_worse'.format(j) for j in range(2, 6) if j != i], axis=1)
    # drop rows with NaN in phase{}_percent
    train_df_new = train_df_new.dropna(subset=['phase{}_worse'.format(i)])
    test_df_new = test_df_new.dropna(subset=['phase{}_worse'.format(i)])
    test_index = test_df_new.index
    
    # LAYER 1: Predict outcome without diff_set features
    X_train_L1 = train_df_new.drop('phase{}_worse'.format(i), axis=1)
    y_train = train_df_new['phase{}_worse'.format(i)]
    X_test_L1 = test_df_new.drop('phase{}_worse'.format(i), axis=1)
    y_test = test_df_new['phase{}_worse'.format(i)]
    
    if i == 3:
        best_params_xgb_regressor = best_params_xgb_regressor_for_p3
    
    # Train Layer 1 model
    model_L1 = xgb.XGBRegressor(**best_params_xgb_regressor)
    model_L1.fit(X_train_L1, y_train)
    
    # Get Layer 1 predictions
    y_pred_L1 = model_L1.predict(X_test_L1)
    
    # Calculate residuals (errors) for training set
    X_train_full = train_df_new.drop('phase{}_worse'.format(i), axis=1)
    y_train_pred_L1 = model_L1.predict(X_train_L1)
    train_residuals = y_train - y_train_pred_L1
    
    # LAYER 2: Predict residuals using diff_set features
    # Extract only diff_set features for Layer 2
    train_now = df_nowcasting[df_nowcasting['date'] < date]
    test_now = df_nowcasting[df_nowcasting['date'] >= date]
    train_now = train_now.drop(['date','area_id'], axis=1)
    test_now = test_now.drop(['date','area_id'], axis=1)
    train_now_new = train_now.drop(['phase{}_percent'.format(j) for j in range(2, 6) if j != i], axis=1)
    test_now_new = test_now.drop(['phase{}_percent'.format(j) for j in range(2, 6) if j != i], axis=1)
    train_now_new = train_now_new.dropna(subset=['phase{}_percent'.format(i)])
    test_now_new = test_now_new.dropna(subset=['phase{}_percent'.format(i)])
    X_train_L2 = train_now_new[diff_set]
    X_test_L2 = test_now_new[diff_set]
    
    # Train Layer 2 model to predict residuals
    model_L2 = xgb.XGBRegressor(**best_params_xgb_regressor)  # You can optimize parameters for this layer too
    model_L2.fit(X_train_L2, train_residuals)
    
    # Get Layer 2 predictions (residual predictions)
    residual_pred = model_L2.predict(X_test_L2)
    
    # Final prediction = Layer 1 prediction + Layer 2 prediction (residuals)
    y_pred_final = y_pred_L1 + residual_pred
    
    # Store results
    y_pred_test = pd.concat([
        y_pred_test, 
        pd.DataFrame({
            'y_pred_L1': y_pred_L1,
            'residual_pred': residual_pred,
            'y_pred': y_pred_final,  # Combined prediction
            'y_test': y_test,
            'phase': [i]*len(y_pred_final),
            'test_index': test_index
        })
    ], ignore_index=True)

# Convert probabilities to phases
y_pred_test = convert_prob_to_phase(y_pred_test)
y_test = y_pred_test['overall_phase']
y_pred = y_pred_test['overall_phase_pred']

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Metrics
accuracy_score_new, sensitivity, precision, overall_r2 = all_metrics(y_test, y_pred, cm, y_pred_test)

print("Accuracy:", accuracy_score_new)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("Overall R²(Phase 3 or more):", overall_r2)



Accuracy: 0.6666666666666666
Sensitivity: 0.9169510807736063
Precision: 0.8035892323030908
Overall R²(Phase 3 or more): 0.27554513043222684


In [7]:
y_pred_test

,y_pred_L1,residual_pred,test_index,y_pred_L1,residual_pred,phase2_pred,phase2_test,test_index,y_pred_L1,residual_pred,...,phase4_pred,phase4_test,test_index,y_pred_L1,residual_pred,phase5_pred,phase5_test,test_index,overall_phase,overall_phase_pred
0,0.0,0.0,0.0,0.646059,-0.000211,0.65,0.50,1,0.264339,-0.004243,...,0.06,0.01,1,0.001075,6.184564e-11,0.0,0.0,1,2.0,3.0
1,0.0,0.0,0.0,0.673405,0.000099,0.67,0.55,2,0.288630,-0.002548,...,0.07,0.02,2,0.001075,6.184564e-11,0.0,0.0,2,2.0,3.0
2,0.0,0.0,0.0,0.652185,-0.000423,0.65,0.80,3,0.278781,-0.002548,...,0.07,0.20,3,0.001075,6.184564e-11,0.0,0.0,3,4.0,3.0
3,0.0,0.0,0.0,0.684710,0.000056,0.68,0.85,6,0.258975,0.007253,...,0.07,0.25,6,0.001075,6.184564e-11,0.0,0.0,6,4.0,3.0
4,0.0,0.0,0.0,0.507810,-0.000034,0.51,0.65,7,0.222396,0.008948,...,0.07,0.01,7,0.001075,6.184564e-11,0.0,0.0,7,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,0.0,0.0,0.0,0.744233,0.000758,0.74,0.65,5562,0.405020,0.006427,...,0.13,0.10,5562,0.001075,6.184564e-11,0.0,0.0,5562,3.0,3.0
1166,0.0,0.0,0.0,0.739506,-0.000385,0.74,0.75,5563,0.395956,-0.002351,...,0.11,0.10,5563,0.001075,6.184564e-11,0.0,0.0,5563,3.0,3.0
1167,0.0,0.0,0.0,0.653362,-0.000504,0.65,0.80,5572,0.371798,0.001000,...,0.13,0.10,5572,0.001075,6.184564e-11,0.0,0.0,5572,3.0,3.0
1168,0.0,0.0,0.0,0.725908,0.000758,0.73,0.65,5573,0.405020,0.006427,...,0.13,0.10,5573,0.001075,6.184564e-11,0.0,0.0,5573,3.0,3.0


In [10]:
nowcasting_prediction = y_pred_test.iloc[:, [-3, -2, -1,-15]]

# keep only area_id, date and overall_phase in df_nowcasting
df_nowcasting = df_nowcasting[['area_id', 'date', 'overall_phase']]
# convert overall_phase in nowcasting_prediction to int
nowcasting_prediction['overall_phase'] = nowcasting_prediction['overall_phase'].astype(int)
# merge nowcasting_prediction and df_nowcasting on test_index and index
df_nowcasting['test_index'] = df_nowcasting.index

merged = pd.merge(nowcasting_prediction,df_nowcasting, on=['test_index','overall_phase'],how='left',indicator=True)


C:\Users\swl00\AppData\Local\Temp\ipykernel_11216\3620574850.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nowcasting_prediction['overall_phase'] = nowcasting_prediction['overall_phase'].astype(int)


In [14]:

# save merged to csv
merged.to_csv(r'C:\Users\swl00\IFPRI Dropbox\Weilun Shi\monitoring-food-crises\New_WorkSpace\1.Source Data\df_vis_nowacast.csv', index=False)